In [6]:
%matplotlib inline

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [8]:
import tensorflow as tf

ModuleNotFoundError: No module named 'tensorflow'

In [4]:
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2024-12-13 20:47:27.989087: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-12-13 20:47:28.003625: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-12-13 20:47:28.003843: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [5]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [6]:
print(tf.reduce_sum(tf.random.normal([1000, 1000])))

2024-12-13 20:47:28.028190: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


tf.Tensor(116.50121, shape=(), dtype=float32)


2024-12-13 20:47:28.031463: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-12-13 20:47:28.031977: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-12-13 20:47:28.032369: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-12-13 20:47:29.183055: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-12-13 20:47:29.183665: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not op

In [7]:
# suppressing the warnings 
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"


In [8]:
print(tf.reduce_sum(tf.random.normal([1000, 1000])))

tf.Tensor(-1013.21124, shape=(), dtype=float32)


In [9]:
import torch

In [10]:
torch.cuda.is_available()

True